In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

from matplotlib import cm
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
path = "/content/drive/My Drive/Thesis/CleanData_withDate.csv"
data = pd.read_csv(path)
#Consider only last 2 columns
data = data[['Admission_D','Discharge_D']]
#Extract only the date part
data[['Admission_D','Discharge_D']] = data[['Admission_D','Discharge_D']].apply(pd.to_datetime)

In [ ]:
#Check for any missing values
data.isnull().sum()

In [ ]:
admission_start = data['Admission_D'].iloc[0]
admission_end = data['Admission_D'].iloc[-1]

discharge_start = data['Discharge_D'].iloc[0]
discharge_end = data['Discharge_D'].iloc[-1]

start = max(admission_start, discharge_start)
end = min(admission_end, discharge_end)

shifted_start = start + timedelta(days=1)
shifted_end = end + timedelta(days=1)

In [ ]:
# TRIALS
print(admission_start)
print(discharge_start)
print(admission_end)
print(discharge_end)
print(start)
print(end)
print(shifted_start)
print(shifted_end)

In [ ]:
date_range = pd.date_range(start=start.date(), end=end.date())
shifted_range = pd.date_range(start=shifted_start.date(), end=shifted_end.date())

In [ ]:
data_per_day = {'Admissions': np.zeros(len(date_range)),
                'Discharges': np.zeros(len(date_range)), 
                'Occupancy':  np.zeros(len(date_range))}               

In [ ]:
i = 0
occupancy_before = 0
for date, shifted_date in zip(date_range, shifted_range):
    cond_1 = date <= data['Admission_D']
    cond_2 = data['Admission_D'] < shifted_date
    data_per_day['Admissions'][i] = len(data[cond_1 & cond_2])
    
    cond_1 = date <= data['Discharge_D']
    cond_2 = data['Discharge_D'] < shifted_date
    data_per_day['Discharges'][i] = len(data[cond_1 & cond_2])
    
    data_per_day['Occupancy'][i] = occupancy_before + data_per_day['Admissions'][i] - data_per_day['Discharges'][i]
    occupancy_before = data_per_day['Occupancy'][i]
    i = i+1

In [ ]:
#Adding the day_of_week,day_of_year,week_of_year and year fields and creating the dataset
df = pd.DataFrame(data_per_day, index = date_range)
#Adding the day of the week part(Monday = 0 and Sunday = 6)
s = date_range.to_series()
day_of_week = s.dt.dayofweek.values
day_of_year = s.dt.dayofyear.values
week_of_year = s.dt.weekofyear.values
year = s.dt.year.values

df['Day_of_week'] = pd.DataFrame(day_of_week, index = date_range) 
df['Day_of_year'] = pd.DataFrame(day_of_year, index = date_range) 
df['Week_of_year'] = pd.DataFrame(week_of_year, index = date_range)
df['Year'] = pd.DataFrame(year, index = date_range)
df['Month_Name'] = pd.DataFrame(s.dt.month_name(), index = date_range)
df['Month'] = pd.DataFrame(s.dt.month.values, index = date_range)

In [ ]:
# Adding Public Holidays
import holidays
# Baden-Württemberg = BW
holidays_BW = holidays.Germany(prov = 'BW', years=range(2002,2016))
# Hessen(Hesse) = HE
holidays_HE = holidays.Germany(prov = 'HE', years=range(2002,2016))
# Rheinland-Pfalz(Rhineland-Palatinate) = RP
holidays_RP = holidays.Germany(prov = 'RP', years=range(2002,2016))

df['P_BW'] = (df.index.isin(holidays_BW)).astype(int)
df['P_H'] = (df.index.isin(holidays_HE)).astype(int)
df['P_RP'] = (df.index.isin(holidays_RP)).astype(int)

In [ ]:
#Removing the 2002 and 2015 data
df_processeddata = df[df['Year'] != 2002] 
df_processeddata = df_processeddata[df_processeddata['Year'] != 2015] 
# df_processeddata.append(schoolholiday)
df_processeddata.to_csv(r'ProcessedDataset.csv',index_label='Date')